# 서약
* 당연한 것을 아래와 같이 한번 다지고 가도록 하겠습니다. 아래 보고서는 **본인의 힘만으로 작성**해야하며, 다른 수강생에게 직접적인 질문, 복사 하는 행위는 모두 금지합니다 
  * 예를 들어서, 본 프로젝트의 코드 셀을 완성하는데 직접적인 질문 또는 복사하는 경우는 모두 금지합니다
  * 수업에서 제공한 코드, 노트북은 모두 재활용가능하며, 카피로 규정하지 않습니다
  * 수업 자료 이외에 참고자료가 있다면, 출처와 사용 부분에 모두 표시하는 경우는 모두 합당한 자료로 인정하겠습니다
  
* 위에 대해서 모두 이해하고 동의했다면, 아래 `서약글`에 다음을 작성해주세요:

"본인은 위 서약글을 이해하고 동의하며, 프로젝트를 수행하는데 있어서 반칙을 할 경우 (제공자 포함) 본 프로젝트에 대한 점수가 반영되지 않는다는 것에 동의 합니다."

**서약서**

이름: 최민기

학번: 20165338

서약글:  본인은 위 서약글을 이해하고 동의하며, 프로젝트를 수행하는데 있어서 반칙을 할 경우(제공자 포함)본 프로젝트에 대한 점수가 반영되지 않는다는 것에 동의합니다.

# Final Project 

## Project: 새 품종 분류하기

## Due date: 2021.06.15

---

* 아래 여러 셀에서 코드를 완성하는 부분을 수행하고, 설명을 요구하는 부분은 보고서에 설명을 최대한 자세하게 적어주세요. 기준은 본인이 이해하고 있다는 것을 표현할 수 있는 부분을 모두 적으시면 됩니다.
  * 답을 작성하는 원칙은 **보고서**를 작성한다고 생각하시면 됩니다
  * 내가 알고 있는 부분을 충실하게 **글로 표현** 하는 것 또한 중요한 연습입니다 
  * 코드 작성은 **주석**으로 설명하시기 바랍니다
  
  
> **제출방법**: 
* 보고서에는 코드 캡쳐 첨부이외에도, 각 코드를 작성하는 방법론과 설명을 작성해야하는 **서술형 문제**도 포함되어 있습니다.
* 서술형 문제는 채점하는 중요한 기준이 됩니다. 성실하게 작성해주세요.
* 서술형 문제에 대한 답변은 첨부된 보고서에 작성하면 됩니다. 


### 목표: 앱을 위한 인공지능 알고리즘 개발 ^ㅡ^
* 본 보고서에서는 모바일/웹앱을 위한 인공지능 알고리즘을 개발하라는 업무를 부여받았다고 가정합니다
* 프로젝트가 완성본은, 사용자가 제공하는 image를 받아서 새의 종(種)을 예측합니다


---

## Step 0: Import library

- 필요하다 생각되는 라이브러리를 미리 import해 놓은 항목입니다.(참고)
- 필요 없는 라이브러리를 제거하거나 필요한 라이브러리를 추가하셔도 됩니다.

In [ ]:
import shutil
import os
import glob
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

from pathlib import Path
from collections import OrderedDict


import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import KFold


In [ ]:
train_on_gpu = torch.cuda.is_available()
#train_on_gpu = False

if not train_on_gpu:
    device = 'cpu'
    print('CUDA is not available.  Training on CPU ...')
else:
    device = 'cuda'
    print('CUDA is available!  Training on GPU ...')

In [ ]:
train_path = "./imgs/train/"  # 데이터 경로 설정
# transform 정규화 하기 전 정확한 mean, std 값 확인하기 위해 사용하는 코드 
# imageFolder로 데이터 불러오기
train_ds = datasets.ImageFolder(root = train_path, transform=transforms.ToTensor())
# 이미지 하나 당 RGB channel별로 mean과 std 계산 -> output은 총 데이터 개수
meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in train_ds]
stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in train_ds]

# 이미지 각 mean과 std의 전체적인 평균 값
meanR = np.mean([m[0] for m in meanRGB])
meanG = np.mean([m[1] for m in meanRGB])
meanB = np.mean([m[2] for m in meanRGB])

stdR = np.mean([s[0] for s in stdRGB])
stdG = np.mean([s[1] for s in stdRGB])
stdB = np.mean([s[2] for s in stdRGB])

print(meanR, meanG, meanB)
print(stdR, stdG, stdB)
# https://deep-learning-study.tistory.com/475 참고

In [ ]:
train_path = "./imgs/train"
test_path = "./imgs/test"

In [ ]:
# data augmentation
train_transform = transforms.Compose([  # augmentation기능들을 순차적으로 시행하도록 묶어줌
    transforms.Resize((240, 240)),  # 모든 이미지의 사이즈를 240*240으로 고정시켜주는 기능
    transforms.RandomRotation (degrees=20),  # Random하게 이미지의 각도를 20으로 회전시키는 기능
    transforms.RandomCrop((224,224)),  # 이미지에서 Random하게 crop(확대)하는 기능
    transforms.RandomHorizontalFlip(p=0.5),  # 이미지를 50%확률로 좌우변환하는 기능
#     transforms.RandomVerticalFlip(p=0.4),  # 이미지를 40%확률로 상하변환하는 기능
    transforms.ToTensor(),  # Tensor로 변환
    transforms.Normalize((0.4734, 0.4809, 0.4165), (0.1831, 0.1783, 0.1856)),  # 이미지 값 정규화
])

valid_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4734, 0.4809, 0.4165), (0.1831, 0.1783, 0.1856)),
])

## Step 3: Neural Network 생성
- Pretrained model을 허용하지 않습니다. (직접 모델을 설계해 주세요)


In [ ]:
#코드작성
#batchnorm은 학습하는파마리터가 있어서 파라미터가 늘어난다.감마랑 베타를 학습
class ConvNet82(nn.Module):  # nn.Module를 상속받는 class 선언
    def __init__(self):   # 자동으로 실행되는 첫번째 메소드인 __init__선언 // self는 자기자신의 인스턴스를 받음
        super().__init__()  # 상속받은 nn.Module의 인스턴스를 임시로 불러오는 부분
        # input : 3*224*224
        # 3*3filter에 padding=1을 주면 이미지의 사이즈가 layer를 적용하기 전의 사이즈와 동일하다.
        # BatchNorm2d는 각 Convolution layer를 거치고 난 후 정규화를 해주어 학습이 더 잘되는 효과를 일으킨다.
        # MaxPool2d는 2*2filter에 stride2로 4개의 픽셀값 중 가장 큰 픽셀값을 가져오고 이미지의 size를 축소시켜 중요한 feature값만 뽑아내는 기능이다.
        # 마지막으로 activation function인 ReLU를 적용하면 음의 값을 가진 데이터가 0으로 수렴한다.
        # 각 conv2d layer를 지날때마다 output의 차원을 다음 layer의 input으로 적용해주어야 한다.
        # ReLU에 True를 넣는 이유 : 메모리를 그 자리에서 직접 연산한다.(복사해서 불러오지 않아도 된다.)
        self.cn1 = nn.Sequential(nn.Conv2d(3, 16, 3, padding=1),nn.BatchNorm2d(16),nn.ReLU(True),
                                nn.Conv2d(16,16,3, padding=1),nn.BatchNorm2d(16),nn.ReLU(True))
        # 모델을 더 딥하게 만들기 위해서 Maxpooling을 하기 전에 Convolution layer를 한개씩 더 추가해 주었다.
        # 16*224*224
        self.cn2 = nn.Sequential(nn.Conv2d(16, 32, 3, padding=1),nn.BatchNorm2d(32),nn.ReLU(True),
                                 nn.Conv2d(32,32,3, padding=1),nn.BatchNorm2d(32),nn.MaxPool2d(2, 2),nn.ReLU(True))
        # 32*112*112
        self.cn3 = nn.Sequential(nn.Conv2d(32, 64, 3, padding=1),nn.BatchNorm2d(64),nn.ReLU(True),
                                 nn.Conv2d(64, 64, 3, padding=1),nn.BatchNorm2d(64),nn.MaxPool2d(2, 2),nn.ReLU(True))
        # 64*56*56
        self.cn4 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1),nn.BatchNorm2d(128),nn.ReLU(True),
                                 nn.Conv2d(128, 128, 3, padding=1),nn.BatchNorm2d(128),nn.MaxPool2d(2, 2),nn.ReLU(True))
        # 128*28*28
        self.cn5 = nn.Sequential(nn.Conv2d(128, 256, 3, padding=1),nn.BatchNorm2d(256),nn.ReLU(True),
                                 nn.Conv2d(256, 256, 3, padding=1),nn.BatchNorm2d(256),nn.MaxPool2d(2, 2),nn.ReLU(True))
        # 256*14*14
        self.cn6 = nn.Sequential(nn.Conv2d(256, 512, 3, padding=1),nn.BatchNorm2d(512),nn.ReLU(True),
                                 nn.Conv2d(512, 512, 3, padding=1),nn.BatchNorm2d(512),nn.MaxPool2d(2, 2),nn.ReLU(True))
        # 512*7*7 = 25088
        
        self.fc1 = nn.Sequential(nn.Linear(25088, 1024), # Convolution layer를 거친 데이터를 1열로 펼쳐서 Fully Connected Layer 적용
                                 nn.BatchNorm1d(1024),   # 1차원이므로 BatchNorm1d를 적용하여 데이터 정규화
                                 nn.ReLU(True),          # activation function 적용
                                 nn.Dropout(0.5),        # 간단하지만 아주 강력한 정규화(regularization) 방법으로 무작위 어떠한 확률로 퍼셉트론을 지워 편향을 억제
                                  
                                 nn.Linear(1024, 300),
                                 nn.BatchNorm1d(300),
                                 nn.ReLU(True),
                                 nn.Dropout(0.5),
                                  
                                 nn.Linear(300, 15))     # 마지막 layer에는 ReLU와 Dropout을 사용하지 않는다.

    def forward(self, x):  # nn.Module에 있는 함수로 ConvNet.forward()를 사용하지 않아도 실행된다. / forward path연산을 하는 함수
        x = self.cn1(x)    # __init__에서 만든 모델들을 순차적으로 적용해준다. 
        x = self.cn2(x)
        x = self.cn3(x)
        x = self.cn4(x)
        x = self.cn5(x)
        x = self.cn6(x)
        x = x.view(x.size(0),-1)  # conv2d layer를 적용시킨 데이터값들을 1차원으로 정렬
        x = self.fc1(x)    # __init__함수에서 선언한 self.fc를 불러와서 만든 모델을 x에 대입

        return x
# model구성 확인
model = ConvNet82()  # ConvNet()모델을 model에 적용

model.to(device)   # model을 gpu사용하도록 설정

 ## Step 4: Cost (Loss) Function 과 Optimizer 선택
 
 [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) 및 [optimizer](http://pytorch.org/docs/stable/optim.html)를 선택하여 코드를 완성하세요.
 
 위 링크에서 다양한 Loss Function과 Optimize Function을 확인 할 수 있습니다
 

---

## Step 5: 구성한 모델에 대한 Train and Validate 진행

* 코드 전체를 주석으로 설명하세요
* Epoch 별로 Loss나 Accuracy를 출력하여 학습 진행 과정을 확인 할 수 있도록 합니다
* 출력 예시는 주어지나 정해진 형식은 없습니다
* 최적의 모델 저장

예제:
```
Started Training...
Epoch: 1 	Training Loss: 3.317162 	Validation Loss: 4.162958
Epoch: 2 	Training Loss: 2.420140 	Validation Loss: 4.182362
...
...
Finished training
```

In [ ]:
class dataset(Dataset):
    def __init__(self,x,y, transform =None):
        self.x = x
        self.y =y
        self.transform = transform
            
    def __getitem__(self,index):
        file_name = self.x[index]
        img = Image.open(file_name).convert('RGB')
        
        label = self.y[index]
        
        if self.transform is not None:
            img = self.transform(img)
            
        return img, label
    
    def __len__(self):
        return len(self.x)

In [ ]:
train_path = './imgs/train/'   # 이미지 경로 설정
dataset = datasets.ImageFolder(root = train_path,) # datasets라이브러리의 ImageFolder함수로 train_path에 있는 데이터 불러옴
classes = dataset.classes # 데이터의 class종류 확인

kf = KFold(n_splits=5, shuffle=True) 
# 데이터를 받으면 5개의 구간으로 나누어 valid셋과 train셋으로 나눠주고 5번fold 반복, 데이터 섞기(shuffle)

def reset_weights(model): # 모델의 weights 값을 초기화 해주는 함수
    for layer in model.children(): # 모델 전체의 모든 module에 대한 iterator를 반환한다.
        if hasattr(layer, 'reset_parameters'): # reset_parameters가 존재한다면 layer의 파라미터 값을 리셋
            layer.reset_parameters()

n_epochs = 100 # epoch를 100으로 설정해 훈련함


for fold, (train_ind, valid_ind) in enumerate(kf.split(dataset)): 
# 만든 kf에 ImageFolder인 dataset를 넣어 train_ind와 valid_ind를 불러옴

    # Fold가 돌때마다 확인
    print('Starting fold = ', fold)
    # for문에서 불러온 index값을 이용해 train과 valid 데이터 분리
    train_data = [dataset[i] for i in train_ind]
    valid_data = [dataset[i] for i in valid_ind]
    #print(train_data[0][0].show())
    
    # 만든 MyDataset class를 사용해 각 데이터의 transform 적용
    train_data_kfold = MyDataset(
       train_data, transform = train_transform)

    valid_data_kfold = MyDataset(
        valid_data, transform = valid_transform)
    #print(valid_data_kfold.__getitem__(0))
    
    # pytorch에서 제공하는 DataLoader를 사용해 미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행가능 하도록 처리
    train_loader_kfold = torch.utils.data.DataLoader(train_data_kfold, batch_size=16, shuffle=True)
    valid_loader_kfold = torch.utils.data.DataLoader(valid_data_kfold, batch_size=8, shuffle=True)
    # model에 ConvNet모델 적용
    model = ConvNet()
    # criterion변수값에 CrossEntropyLoss()값 불러오기
    
    criterion = nn.CrossEntropyLoss()
    # optim의 Adam optimizer를 사용
    # Adam은 가장 많이 쓰이는 optimizer 중 하나로 Momentum과 RMSProp를 융합한 방법이다. 
    optimizer = optim.Adam(model.parameters(), lr = 8e-5)
    # scheduler는 StepLR를 사용하여 epoch을 40번 할때마다 lr의 값을 줄여주어 정확도의 최적값에 세밀하게 도달하는 효과를 준다.
    scheduler = lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)
    # fold를 돌때마다 model의 weigths값 초기화
    model.apply(reset_weights)

    valid_acc_max = 0.0 # valid 최고 정확도 값 초기설정

    # loss값과 acc값을 저장하는 변수 생성
    train_loss = torch.zeros(n_epochs)
    valid_loss = torch.zeros(n_epochs)

    train_acc = torch.zeros(n_epochs)
    valid_acc = torch.zeros(n_epochs)
    
    # 훈련 for문 
    for e in range(0, n_epochs):
        model.train()   # model의 weight값들을 훈련하여 바뀌도록 설정
        for data, labels in tqdm(train_loader_kfold): # tqdm을 사용하여 batch를 읽는 위치 확인
            #만든 train dataloader 데이터셋 읽기
            model.to(device) # gpu할당
            data, labels = data.to(device), labels.to(device) # gpu할당

            optimizer.zero_grad() # 이전에 계산된 기울기(미분값) 초기화
            # forward pass
            logits = model(data) # 데이터 모델 적용 / logit 값
            
            loss = criterion(logits, labels) # cross_entropy loss를 사용해 오차값 loss에 넣어줌
            # backward pass
            loss.backward() # 편미분을 통해 weights값을 갱신
            
            optimizer.step() # parameter 적용(업데이트)
            # loss값 저장
            train_loss[e] += loss.item()
            
            ps = F.softmax(logits, dim=1) # softmax를 사용해 logits값을 확률값으로 변환

            top_class = torch.argmax(ps, dim=1) # 같은 행중에서 확률 값이 가장 큰 값을 예측값으로 결정

            equals = top_class == labels.reshape(top_class.shape) # 예측한 값이 실제 값과 맞는지 확인
            train_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu() # 정답을 맞춘 정확도 확인

        train_loss[e] /= len(train_loader_kfold) # 데이터의 loss들의 평균
        train_acc[e] /= len(train_loader_kfold) # 데이터의 acc들의 평균


        # validation
        with torch.no_grad(): # gradient값 고정
            model.eval() # 평가 모드 / dropout을 하지 않는다.
            for data, labels in tqdm(valid_loader_kfold):

                data, labels = data.to(device), labels.to(device)
                # forward pass
                logits = model(data)
                # loss 계산
                loss = criterion(logits, labels)
                # valid loss 값
                valid_loss[e] += loss.item()
                # softmax 적용으로 logits값을 확률값으로 변환
                ps = F.softmax(logits, dim=1)
                # argmax로 확률 값 중 가장 큰 값을 예측값으로 설정
                top_class = torch.argmax(ps, dim=1)
                equals = top_class == labels.reshape(top_class.shape)
                valid_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()

        # loss와 acc의 평균 값 
        valid_loss[e] /= len(valid_loader_kfold)
        valid_acc[e] /= len(valid_loader_kfold)

        # loss 출력
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            e, train_loss[e], valid_loss[e]))

        # accuracy 출력
        print('Epoch: {} \tTraining accuracy: {:.6f} \tValidation accuracy: {:.6f}'.format(
            e, train_acc[e], valid_acc[e]))

        # valid데이터의 accuracy 값이 가장 높은 모델 저장
        if valid_acc[e] >= valid_acc_max:
            print('Validation acc decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_acc_max,
            valid_acc[e]))
            # 각 fold마다 모델을 저장하기 위해 if문 사용
            if fold == 0:
                torch.save(model.state_dict(), 'model0.pt')
            elif fold == 1:
                torch.save(model.state_dict(), 'model1.pt')
            elif fold == 2:
                torch.save(model.state_dict(), 'model2.pt')
            elif fold == 3:
                torch.save(model.state_dict(), 'model3.pt')
            else:
                torch.save(model.state_dict(), 'model4.pt')
            valid_acc_max = valid_acc[e]
        print('best accuracy ({:.6f})'.format(valid_acc_max))
        # Learning rate값에 scheduler 적용
        scheduler.step()

## Step 6: CNN model training/validation 분석
   * tranining loss와 validation loss 그래프를 통해서 분석


In [ ]:
# Kfold로 모델을 훈련하고 train과 valid에 대한 loss변화에 대한 시각적 분석
# plt.plot(train_loss)
# plt.plot(valid_loss)

In [ ]:
# Kfold로 모델을 훈련하고 train과 valid에 대한 accuracy변화에 대한 시각적 분석
# plt.plot(train_acc)
# plt.plot(valid_acc)

In [ ]:
# 저장된 파라미터값을 각 모델에 적용
# gpu할당
# eval모드로 설정하여 dropout을 하지 않음
model0.load_state_dict(torch.load('model0.pt'))
model0.to(device)
model0.eval()

model1.load_state_dict(torch.load('model1.pt'))
model1.to(device)
model1.eval()

model2.load_state_dict(torch.load('model2.pt'))
model2.to(device)
model2.eval()

model3.load_state_dict(torch.load('model3.pt'))
model3.to(device)
model3.eval()

model4.load_state_dict(torch.load('model4.pt'))
model4.to(device)
model4.eval()

model79_1.load_state_dict(torch.load('model79_1.pt'))
model79_1.to(device)
model79_1.eval()

model79_2.load_state_dict(torch.load('model79_2.pt'))
model79_2.to(device)
model79_2.eval()

model79_3.load_state_dict(torch.load('model79_3.pt'))
model79_3.to(device)
model79_3.eval()

## Step 7: Predict with Test Data 


### 예시 코드를 제공해 드립니다. 필요한 부분을 채워 사용하시거나 직접 코드를 작성 하셔도 됩니다

In [ ]:
# test데이터의 전처리 과정 Dataset을 input으로 받음
class test_dataset(Dataset):
    def __init__(self,imgpath,transform=None): # 이미지 경로와 transform을 받아옴
        
        self.imgpath = imgpath # self로 값을 받음
        self.transform = transform
        
    def __len__(self): # self 적용
        return len(self.imgpath) # 받은 데이터의 길이 return
    
    def __getitem__(self,idx): # image의 idx로 순서대로 불러와 RGB 데이터 transform적용
        x = self.transform(Image.open(self.imgpath[idx]).convert('RGB'))
        
        return x

In [ ]:
# valid transform과 동일하게 적용
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4734, 0.4809, 0.4165), (0.1831, 0.1783, 0.1856)),
    # 각 데이터의 mean과 std 정규화
])

### test dataset과 dataloader 생성


In [ ]:
# print(test_set)

In [ ]:
# test데이터를 순서대로 불러옴
test_set = sorted(glob.glob('./imgs/test/*'))
test_batch = 1
# transform 적용
test_data = test_dataset(test_set,test_transform)
# test의 순서롤 섞지않고 batch를 1로 주어 이미지를 하나씩 주는 dataloader
test_loader = torch.utils.data.DataLoader(test_data, batch_size=test_batch, shuffle=False, num_workers=0)

In [ ]:
print(len(test_data)) # test_set이 잘 들어왔나 확인

In [ ]:
# images, labels = next(iter(test_loader))
# labels

In [ ]:
pred = []

for i, data in enumerate(test_loader):

    # 데이터를 list로 받음
    inputs = []
    inputs.append(data)
    # stack으로 순서대로 데이터를 받고 gpu할당
    inputs = torch.stack(inputs).to(device)
    data = data.to(device)
    # 각 모델에 data를 넣어 logit값을 뽑음
    logit0 = model0(data)
    logit1 = model1(data)
    logit2 = model2(data)
    logit3 = model3(data)
    logit4 = model4(data)
    logit79_1 = model79_1(data)
    logit79_2 = model79_2(data)
    logit79_3 = model79_3(data)
    # 앙상블의 voting 방법을 사용하여 전체 logit값의 평균을 정함
    logit = (logit0+logit1+logit2+logit3+logit4+logit79_1+logit79_2+logit79_3)/8
    # 앙상블한 logit값에 softmax를 적용해 확률값으로 만듦
    ps = F.softmax(logit)   
    # 확률 값 중 가장 큰 값을 예측 값으로 결정
    top_class = torch.argmax(ps, dim=1)  
    # pred에 예측한 값을 testset순서대로 넣어줌
    pred += [top_class.detach().cpu().numpy()]   
    del logit   # logit값 초기화
print(ps.shape)
print(pred[:10])


In [ ]:
# pred의 값을 numpy array로 변환, int32 type로 바꿔줌
pred1 = np.array(pred,dtype=np.int32)
# 1차원으로 reshape
pred1 = pred1.reshape(-1)
pred1.shape

In [ ]:
#예측 값 확인
pred1[:10]

### Predict 수행

In [ ]:
# test 데이터의 label값을 classes변수에 넣어줌
classes = [i.split('\\')[-1] for i in glob.glob('./imgs/train/*')]

In [ ]:
#클래스 종류 정렬 확인
classes

In [ ]:
# csv 파일 만드는 함수
id=[]
category =[]
for i in range(len(test_set)):
    id.append(test_set[i].split('\\')[-1])
    category.append(classes[pred1[i]])
pd.DataFrame({'Id':id,'Category':category}).to_csv('submission.csv',index=False)

예측 결과 인덱스를 저장한 pred 값을 사용해 클래스를 매칭하기 위한 class_name을 담아 놓은 리스트입니다.

주의할 점은 Train과정에서 사용된 class lable값과 같은 순서로 저장이 되어 있어야 한다는 점입니다
